# 실험 06-01 BJT 바이어스 회로 시뮬레이션

## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import io
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from numpy import nan, inf

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def derivative(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return d

def get_simulation_result(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1
    
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1

    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def print_value_to_string(label, value):
    output = io.StringIO()
    print('%s = ' % label, end='', file=output)
    print('{:11.3f}'.format(value), file=output)
    captured = output.getvalue()
    return captured

def draw_plot(xs, ys, label, style_idx, color_idx=-1, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if color_idx < 0:
        color_idx = style_idx

    if isinstance(xs, list):
        xs = np.array(xs)
    if isinstance(ys, list):
        ys = np.array(ys)

    if (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    elif not isinstance(xs, np.ndarray):
        selected_markevery = None
        selected_marker = markers[style_idx]
    elif (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[color_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[color_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

## LTspice 명령

* 컴포넌트 추가 명령
    + Voltage source: Draft -> Component -> voltage (V)
    + Resister: Draft -> Component -> res (R)
    + Diode: Draft -> Component -> diode (D)
    + Capacitor: Draft -> Component -> cap (C)
    + Zener diode: Draft -> Component -> zenor
    + Bipolar NPN transistor: Draft -> Component -> npn
    + Bipolar PNP transistor: Draft -> Component -> pnp
    + N-Channel MOSFET transistor: Draft -> Component -> nmos
    + Ground: (G)
    + SPICE 시뮬레이션 설정 명령: Draft -> SPICE directive (S)
    + Net name: Draft ->  Net Name (F4)
    + Comment text: Draft -> Comment Text (T)

&nbsp;

* SPICE 시뮬레이션 설정 명령
    * DC Sweep analysis (start and stop value)
        + Compute the DC operating point over swept independent source(s).
        + Syntax: .dc \<source1\> \<start1\> \<stop1\> [\<increment1\>] [\<source2\> ...]
            + source: source name
            + start: start value
            + stop: stop value
            + increment: increment value
    * DC Sweep analysis (list)
        + Compute the DC operating point over swept independent source(s).
        + Syntax: .dc \<source1\> list \<value1\> [\<value2\> [\<value3\> [...]]] [\<source2\> ...]
            + source: source name
    * Transient analysis
        + Performa time-domain analysis
        + Syntax: .tran \<Tprint\> \<Tstop\> [\<Tstart\> [\<Tmaxstep\>]]
            - Tprint: printing increment (Use 0 in LTspice)
            - Tstop: stop time
            - Tstart: time to start saving waveform data
            - Tmaxstep: maximum timestep size
    * Parametric analysis (start and stop value)
        + Performs a parametric sweep for all of the analysis
        + Syntax: .step param \<name\> \<start\> \<stop\> \<step\>
            - name: name of parameter to step
            - start: start value
            - stop: stop value
            - step: value step
    * Parametric analysis (list)
        + Performs a parametric sweep for all of the analysis
        + Syntax: .step param \<name\> list \<value1\> [\<value2\> [\<value3\> [...]]]
            - name: name of parameter to step
    + DC Bias Point  analysis
        + Performa DC bias point analysis
        + Syntax: .op

&nbsp;

* 기타 명령
    + 회전 (Control + R)
    + Wire (F3)
    + Move (F7)
    + Drag (F8)
    + Delete (F5)
    + Duplicate (F6)


*******************************************************************************
## 시뮬레이션 1-1 결과 분석

## 회로도

* 사용한 컴포넌트
    + Voltage source: VCC
    + Ground
    + Resister: R1, R2, RC
    + Bipolar NPN transistor : Q1
    + Net Name: VB, VC

![Markdown-mark](../ltspice/simulation_01_01.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_01_01.txt')

### 2. 시뮬레이션 결과에서 $I_B$, $I_C$, $\beta_{DC}$, $V_B$, $V_C$, $V_{BE}$, $V_{CE}$, $V_{CB}$를 가져온다.

In [ ]:
R_C_kOhm = data['RCVAR'] / 1000000.
I_B_mA   = data['IB(Q1)']
I_C_mA   = data['IC(Q1)']
V_B_V    = data['V(VB)'] / 1000.
V_C_V    = data['V(VC)'] / 1000.

Beta_DC  = I_C_mA / I_B_mA
V_BE_V   = V_B_V
V_CE_V   = V_C_V
V_CB_V   = V_C_V - V_B_V

VCC_V = 10.

print_array('R_C_kOhm', R_C_kOhm)
print_array('I_B_mA  ', I_B_mA)
print_array('I_C_mA  ', I_C_mA)
print_array('Beta_DC ', Beta_DC)
print_array('V_BE_V  ', V_BE_V)
print_array('V_CE_V  ', V_CE_V)
print_array('V_CB_V  ', V_CB_V)

### 3. $V_{CE}$ - $I_C$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 10.5])
plt.ylim([-1, 22])
for idx in range(len(R_C_kOhm)):
    draw_plot(V_CE_V[idx], I_C_mA[idx], '(R_C=%1.3fkOhm)' % R_C_kOhm[idx], idx, scatter=True, scatter_s=20)
plt.show()

### 4. 부하선을 그린다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 10.5])
plt.ylim([-1, 22])
for idx in range(len(R_C_kOhm)):
    draw_plot([0, VCC_V], [VCC_V/R_C_kOhm[idx], 0], None, idx, marker_num = 0)
    draw_plot(V_CE_V[idx], I_C_mA[idx], '(R_C=%1.3fkOhm)' % R_C_kOhm[idx], idx, scatter=True, scatter_s=20)
plt.show()

*******************************************************************************
## 시뮬레이션 1-2 결과 분석

## 회로도

* 사용한 컴포넌트
    + Voltage source: VBE, VCE
    + Ground
    + Bipolar NPN transistor : Q1
    + Net Name: VB, VC

![Markdown-mark](../ltspice/simulation_01_02.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_01_02.txt')

### 2. 시뮬레이션 결과에서 $I_C$, $V_{CE}$를 가져온다.

In [ ]:
I_C_mA_dc_sweep = []
V_CE_V_dc_sweep = []
labels = [0.712, 0.712, 0.712, 0.711, 0.706, 0.701] # VBE

for idx in range(len(labels)):
    I_C_mA_dc_sweep.append(data['(IC(Q1))@%d' % (idx+1)])
    V_CE_V_dc_sweep.append(data['(V(VC))@%d' % (idx+1)] / 1000.)

### 3. $V_{CE}$ - $I_C$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 6.5])
plt.ylim([-0.5, 7.5])
for idx in range(len(labels)):
    draw_plot(V_CE_V_dc_sweep[idx], I_C_mA_dc_sweep[idx], '(V_BE=%1.3fV)' % labels[idx], idx, marker_num=0)
plt.show()

### 4. 시뮬레이션 1-1의 결과와 비교한다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 10.5])
plt.ylim([-1, 7])
for idx in range(len(R_C_kOhm)):
    draw_plot([0, VCC_V], [VCC_V/R_C_kOhm[idx], 0], None, idx, marker_num = 0)
    draw_plot(V_CE_V[idx], I_C_mA[idx], '(R_C=%1.3fkOhm)' % R_C_kOhm[idx], idx, scatter=True, scatter_s=20)
for idx in range(len(labels)):
    draw_plot(V_CE_V_dc_sweep[idx], I_C_mA_dc_sweep[idx], '(V_BE=%1.3fV)' % labels[idx], idx+0, marker_num=0)
plt.show()

*******************************************************************************
## 시뮬레이션 2-1 결과 분석

## 회로도

* 사용한 컴포넌트
    + Voltage source: VCC
    + Ground
    + Resister: R1, R2, RC
    + Bipolar PNP transistor : Q1
    + Net Name: VB, VC

![Markdown-mark](../ltspice/simulation_02_01.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_02_01.txt')

### 2. 시뮬레이션 결과에서 $I_B$, $I_C$, $V_B$, $V_C$, $\beta_{DC}$, $V_{BE}$, $V_{CE}$, $V_{CB}$를 가져온다.

In [ ]:
R_C_kOhm = data['RCVAR'] / 1000000.
I_B_mA   = data['IB(Q1)']
I_C_mA   = data['IC(Q1)']
V_B_V    = data['V(VB)'] / 1000.
V_C_V    = data['V(VC)'] / 1000.

Beta_DC  = I_C_mA / I_B_mA
V_BE_V   = V_B_V
V_CE_V   = V_C_V
V_CB_V   = V_C_V - V_B_V

VCC_V = -10.

print_array('R_C_kOhm', R_C_kOhm)
print_array('I_B_mA  ', I_B_mA)
print_array('I_C_mA  ', I_C_mA)
print_array('Beta_DC ', Beta_DC)
print_array('V_BE_V  ', V_BE_V)
print_array('V_CE_V  ', V_CE_V)
print_array('V_CB_V  ', V_CB_V)

### 3. $V_{CE}$ - $I_C$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-10.5, 0.5])
plt.ylim([-22, 1])
for idx in range(len(R_C_kOhm)):
    draw_plot(V_CE_V[idx], I_C_mA[idx], '(R_C=%1.3fkOhm)' % R_C_kOhm[idx], idx, scatter=True, scatter_s=20)
plt.show()

### 4. 부하선을 그린다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-10.5, 0.5])
plt.ylim([-22, 1])
for idx in range(len(R_C_kOhm)):
    draw_plot([VCC_V, 0], [0, VCC_V/R_C_kOhm[idx]], None, idx, marker_num = 0)
    draw_plot(V_CE_V[idx], I_C_mA[idx], '(R_C=%1.3fkOhm)' % R_C_kOhm[idx], idx, scatter=True, scatter_s=20)
plt.show()

*******************************************************************************
## 시뮬레이션 2-2 결과 분석

## 회로도

* 사용한 컴포넌트
    + Voltage source: VBE, VCE
    + Ground
    + Bipolar PNP transistor : Q1
    + Net Name: VB, VC

![Markdown-mark](../ltspice/simulation_02_02.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_02_02.txt')

### 2. 시뮬레이션 결과에서 $I_C$, $V_{CE}$를 가져온다.

In [ ]:
I_C_mA_dc_sweep = []
V_CE_V_dc_sweep = []
labels = [-0.711, -0.711, -0.710, -0.701, -0.696, -0.691] # VBE

for idx in range(len(labels)):
    I_C_mA_dc_sweep.append(data['(IC(Q1))@%d' % (idx+1)])
    V_CE_V_dc_sweep.append(data['(V(VC))@%d' % (idx+1)] / 1000.)

### 3. $V_{CE}$ - $I_C$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-6.5, 0.5])
plt.ylim([-9.5, 0.5])
for idx in range(len(labels)):
    draw_plot(V_CE_V_dc_sweep[idx], I_C_mA_dc_sweep[idx], '(V_BE=%1.3fV)' % labels[idx], idx, marker_num=0)
plt.show()

### 4. 시뮬레이션 2-1의 결과와 비교한다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-10.5, 0.5])
plt.ylim([-10, 1])
for idx in range(len(R_C_kOhm)):
    draw_plot([VCC_V, 0], [0, VCC_V/R_C_kOhm[idx]], None, idx, marker_num = 0)
    draw_plot(V_CE_V[idx], I_C_mA[idx], '(R_C=%1.3fkOhm)' % R_C_kOhm[idx], idx, scatter=True, scatter_s=20)
for idx in range(len(labels)):
    draw_plot(V_CE_V_dc_sweep[idx], I_C_mA_dc_sweep[idx], '(V_BE=%1.3fV)' % labels[idx], idx, marker_num=0)
plt.show()

*******************************************************************************
## 시뮬레이션 3-1 결과 분석

## 회로도

* 사용한 컴포넌트
    + Voltage source: VCC
    + Ground
    + Resister: R1, R2, RC, RE
    + Bipolar NPN transistor : Q1
    + Net Name: VB, VC

![Markdown-mark](../ltspice/simulation_03_01.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_03_01.txt')

### 2. 시뮬레이션 결과에서 $I_B$, $I_C$, $\beta_{DC}$, $V_B$, $V_E$, $V_C$, $V_{BE}$, $V_{CE}$, $V_{CB}$를 가져온다.

In [ ]:
R_E_kOhm = data['REVAR'] / 1000000.
I_B_mA   = data['IB(Q1)']
I_C_mA   = data['IC(Q1)']
V_B_V    = data['V(VB)'] / 1000.
V_C_V    = data['V(VC)'] / 1000.
V_E_V    = data['V(VE)'] / 1000.

Beta_DC  = I_C_mA / I_B_mA
V_BE_V   = V_B_V - V_E_V
V_CE_V   = V_C_V - V_E_V
V_CB_V   = V_C_V - V_B_V

VCC_V = 10.
R_C_kOhm = 1.8

print_array('R_E_kOhm', R_E_kOhm)
print_array('I_B_mA  ', I_B_mA)
print_array('I_C_mA  ', I_C_mA)
print_array('Beta_DC ', Beta_DC)
print_array('V_B_V   ', V_B_V)
print_array('V_E_V   ', V_E_V)
print_array('V_C_V   ', V_C_V)
print_array('V_BE_V  ', V_BE_V)
print_array('V_CE_V  ', V_CE_V)
print_array('V_CB_V  ', V_CB_V)

### 3. $V_{CE}$ - $I_C$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 10.5])
plt.ylim([-1, 6])
for idx in range(len(R_E_kOhm)):
    draw_plot(V_CE_V[idx], I_C_mA[idx], '(R_E=%1.3fkOhm)' % R_E_kOhm[idx], idx, scatter=True, scatter_s=20)
plt.show()

### 4. 부하선을 그린다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 10.5])
plt.ylim([-1, 6])
for idx in range(len(R_E_kOhm)):
    draw_plot([VCC_V, 0], [0, VCC_V/(R_C_kOhm + R_E_kOhm[idx])], None, idx, marker_num = 0)
    draw_plot(V_CE_V[idx], I_C_mA[idx], '(R_E=%1.3fkOhm)' % R_E_kOhm[idx], idx, scatter=True, scatter_s=20)
plt.show()

*******************************************************************************
## 시뮬레이션 3-2 결과 분석

## 회로도

* 사용한 컴포넌트
    + Voltage source: VBE, VCE
    + Ground
    + Bipolar NPN transistor : Q1
    + Net Name: VB, VC

![Markdown-mark](../ltspice/simulation_03_02.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_03_02.txt')

### 2. 시뮬레이션 결과에서 $I_C$, $V_{CE}$를 가져온다.

In [ ]:
V_BE_V_dc_sweep = np.array([0.709, 0.695, 0.686, 0.677, 0.667, 0.662])
V_CE_V_dc_sweep = []
I_C_mA_dc_sweep = []

for idx in range(len(V_BE_V_dc_sweep)):
    V_CE_V_dc_sweep.append(data['(V(VC))@%d' % (idx+1)] / 1000.)
    I_C_mA_dc_sweep.append(data['(IC(Q1))@%d' % (idx+1)])

### 3. $V_{CE}$ - $I_C$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 10.5])
plt.ylim([-1, 6])
for idx in range(len(V_BE_V_dc_sweep)):
    draw_plot(V_CE_V_dc_sweep[idx], I_C_mA_dc_sweep[idx], '(V_BE=%1.3fV)' % V_BE_V_dc_sweep[idx], idx, marker_num=0)
plt.show()

### 4. 시뮬레이션 3-1의 결과와 비교한다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 10.5])
plt.ylim([-1, 6])
for idx in range(len(R_E_kOhm)):
    draw_plot([VCC_V, 0], [0, VCC_V/(R_C_kOhm + R_E_kOhm[idx])], None, idx, marker_num = 0)
    draw_plot(V_CE_V[idx], I_C_mA[idx], '(R_E=%1.3fkOhm)' % R_E_kOhm[idx], idx, scatter=True, scatter_s=20)
for idx in range(len(V_BE_V_dc_sweep)):
    draw_plot(V_CE_V_dc_sweep[idx], I_C_mA_dc_sweep[idx], '(V_BE=%1.3fV)' % V_BE_V_dc_sweep[idx], idx, marker_num=0)
plt.show()

*******************************************************************************
## 시뮬레이션 4-1 결과 분석

## 회로도

* 사용한 컴포넌트
    + Voltage source: VCC
    + Ground
    + Resister: R1, R2, RC, RE
    + Bipolar PNO transistor : Q1
    + Net Name: VB, VC

![Markdown-mark](../ltspice/simulation_04_01.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_04_01.txt')

### 2. 시뮬레이션 결과에서 $I_B$, $I_C$, $\beta_{DC}$, $V_B$, $V_E$, $V_C$, $V_{BE}$, $V_{CE}$, $V_{CB}$를 가져온다.

In [ ]:
R_E_kOhm = data['REVAR'] / 1000000.
I_B_mA   = data['IB(Q1)']
I_C_mA   = data['IC(Q1)']
V_B_V    = data['V(VB)'] / 1000.
V_C_V    = data['V(VC)'] / 1000.
V_E_V    = data['V(VE)'] / 1000.

Beta_DC  = I_C_mA / I_B_mA
V_BE_V   = V_B_V - V_E_V
V_CE_V   = V_C_V - V_E_V
V_CB_V   = V_C_V - V_B_V

VCC_V = -10.
R_C_kOhm = 1.8

print_array('R_E_kOhm', R_E_kOhm)
print_array('I_B_mA  ', I_B_mA)
print_array('I_C_mA  ', I_C_mA)
print_array('Beta_DC ', Beta_DC)
print_array('V_B_V   ', V_B_V)
print_array('V_E_V   ', V_E_V)
print_array('V_C_V   ', V_C_V)
print_array('V_BE_V  ', V_BE_V)
print_array('V_CE_V  ', V_CE_V)
print_array('V_CB_V  ', V_CB_V)

### 3. $V_{CE}$ - $I_C$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-10.5, 0.5])
plt.ylim([-6, 1])
for idx in range(len(R_E_kOhm)):
    draw_plot(V_CE_V[idx], I_C_mA[idx], '(R_E=%1.3fkOhm)' % R_E_kOhm[idx], idx, scatter=True, scatter_s=20)
plt.show()

### 4. 부하선을 그린다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-10.5, 0.5])
plt.ylim([-6, 1])
for idx in range(len(R_E_kOhm)):
    draw_plot([VCC_V, 0], [0, VCC_V/(R_C_kOhm + R_E_kOhm[idx])], None, idx, marker_num = 0)
    draw_plot(V_CE_V[idx], I_C_mA[idx], '(R_E=%1.3fkOhm)' % R_E_kOhm[idx], idx, scatter=True, scatter_s=20)
plt.show()

*******************************************************************************
## 시뮬레이션 4-2 결과 분석

## 회로도

* 사용한 컴포넌트
    + Voltage source: VBE, VCE
    + Ground
    + Bipolar PNP transistor : Q1
    + Net Name: VB, VC

![Markdown-mark](../ltspice/simulation_04_02.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_04_02.txt')

### 2. 시뮬레이션 결과에서 $I_C$, $V_{CE}$를 가져온다.

In [ ]:
V_BE_V_dc_sweep = np.array([-0.696, -0.680, -0.672, -0.664, -0.654, -0.650])

V_CE_V_dc_sweep = []
I_C_mA_dc_sweep = []

for idx in range(len(V_BE_V_dc_sweep)):
    V_CE_V_dc_sweep.append(data['(V(VC))@%d' % (idx+1)] / 1000.)
    I_C_mA_dc_sweep.append(data['(IC(Q1))@%d' % (idx+1)])

### 3. $V_{CE}$ - $I_C$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-10.5, 0.5])
plt.ylim([-6, 1])
for idx in range(len(V_BE_V_dc_sweep)):
    draw_plot(V_CE_V_dc_sweep[idx], I_C_mA_dc_sweep[idx], '(V_BE=%1.3fV)' % V_BE_V_dc_sweep[idx], idx, marker_num=0)
plt.show()

### 4. 시뮬레이션 3-1의 결과와 비교한다.

In [ ]:
plt.xlabel('V_CE_V')
plt.ylabel('I_C_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-10.5, 0.5])
plt.ylim([-6, 1])
for idx in range(len(R_E_kOhm)):
    draw_plot([VCC_V, 0], [0, VCC_V/(R_C_kOhm + R_E_kOhm[idx])], None, idx, marker_num = 0)
    draw_plot(V_CE_V[idx], I_C_mA[idx], '(R_E=%1.3fkOhm)' % R_E_kOhm[idx], idx, scatter=True, scatter_s=20)
for idx in range(len(V_BE_V_dc_sweep)):
    draw_plot(V_CE_V_dc_sweep[idx], I_C_mA_dc_sweep[idx], '(V_BE=%1.3fV)' % V_BE_V_dc_sweep[idx], idx, marker_num=0)
plt.show()